# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [50]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, make_scorer
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier,LogisticRegression
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [51]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [52]:
# word net installation:

# unmark if you want to use and need to install
!pip install wn
!python -m wn download omw-he:1.4


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip

Cached file found: C:\Users\tomer\.wn_data\downloads\7ecf10e89326bc0ac26ad94b40fe60a7b6ac3302

Checking C:\Users\tomer\AppData\Local\Temp\tmpl9u7d_tq\omw-he\omw-he.xml
Skipping omw-he:1.4 (Hebrew Wordnet); already added



In [53]:
# word net import:

# unmark if you want to use:
import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [54]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
!pip install hebrew_tokenizer


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [55]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [56]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [57]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [58]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

This code prepares text data for machine learning analysis by creating a custom tokenizer function that removes punctuation and digits and tokenizes the text. After tokenization the data is passed to a vectorizer that is found using a grid search and cross validation.

In [59]:
# preprocessing
def custom_tokenizer(text):
    punctuation_chars = "!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~"
    clean_text = ""

    for char in text:
        if char not in punctuation_chars and not char.isdigit():
            clean_text += char

    text = clean_text
    tokens = ht.tokenize(text)
    return [str(tuple[1]) for tuple in tokens]

# prepare the data
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
X = df_train.story
y = df_train.gender


 # perform cross-validation with the current vectorizer and hyperparameters

f1_scorer = make_scorer(f1_score, greater_is_better=True, average='macro')

vectorizers = [
    (CountVectorizer, {'min_df': [0, 0.05, 0.1], 'max_df': [0.5, 0.7,0.95], 'ngram_range': [(1, 1), (2, 4)]}),
    (TfidfVectorizer, {'min_df': [0, 0.05, 0.1], 'max_df': [0.5, 0.7,0.95], 'ngram_range': [(1, 1), (2, 4)]}),
]


kf = KFold(n_splits=5, shuffle=True, random_state=42)
lr = LogisticRegression(max_iter=1000)

for VectorizerClass, vectorizer_params in vectorizers:
    for min_df in vectorizer_params['min_df']:
        for max_df in vectorizer_params['max_df']:
            for ngram_range in vectorizer_params['ngram_range']:
                vec = VectorizerClass(min_df=min_df, max_df=max_df, ngram_range=ngram_range, tokenizer=custom_tokenizer)
                X_transformed = vec.fit_transform(X)
                lr_scores = cross_val_score(lr, X_transformed, y, cv=kf, scoring=f1_scorer)
                mean_score = np.mean(lr_scores)

                print(f"Vectorizer: {VectorizerClass.__name__}, min_df: {min_df}, max_df: {max_df}, ngram_range: {ngram_range}, F1-score: {mean_score}")


Vectorizer: CountVectorizer, min_df: 0, max_df: 0.5, ngram_range: (1, 1), F1-score: 0.582191845529447
Vectorizer: CountVectorizer, min_df: 0, max_df: 0.5, ngram_range: (2, 4), F1-score: 0.47856280728711365
Vectorizer: CountVectorizer, min_df: 0, max_df: 0.7, ngram_range: (1, 1), F1-score: 0.5962704925033334
Vectorizer: CountVectorizer, min_df: 0, max_df: 0.7, ngram_range: (2, 4), F1-score: 0.47856280728711365
Vectorizer: CountVectorizer, min_df: 0, max_df: 0.95, ngram_range: (1, 1), F1-score: 0.5971835669362429
Vectorizer: CountVectorizer, min_df: 0, max_df: 0.95, ngram_range: (2, 4), F1-score: 0.47856280728711365
Vectorizer: CountVectorizer, min_df: 0.05, max_df: 0.5, ngram_range: (1, 1), F1-score: 0.6175087212731226
Vectorizer: CountVectorizer, min_df: 0.05, max_df: 0.5, ngram_range: (2, 4), F1-score: 0.5305775360831575
Vectorizer: CountVectorizer, min_df: 0.05, max_df: 0.7, ngram_range: (1, 1), F1-score: 0.6219973164857018
Vectorizer: CountVectorizer, min_df: 0.05, max_df: 0.7, ngra

After the pre-processing model are decided, we use a TfidfVectorizer to transform the text data into numerical features, and the resulting sparse matrix is converted into a pandas DataFrame. from the original train data we generate 2 DataFrames, and the gender column is transformed into binary labels, once the female is '1' and once the male is '1'

In [60]:
# prepare the data
vec = TfidfVectorizer(max_df=0.95, min_df=0, ngram_range=(1, 2), max_features=40000, tokenizer=custom_tokenizer)
X_train = vec.fit_transform(df_train["story"])
X_train_CV = pd.DataFrame(X_train.toarray(), columns=vec.get_feature_names_out())

# spliting the data
X_train_CV["gender"] = df_train["gender"]
X_train_male = X_train_CV.copy()
X_train_female = X_train_CV.copy()

X_train_male["gender"] = np.where(X_train_male["gender"] == 'f',0,1)
X_train_female["gender"] = np.where(X_train_female["gender"] == 'f',1,0)

target_columns = X_train_CV.columns[X_train_CV.columns != "gender"]
X_male = X_train_male[target_columns]
X_female = X_train_female[target_columns]

y_male = X_train_male["gender"]
y_female = X_train_female["gender"]




Logistic Regression is a type of machine learning algorithm used for binary classification tasks, where the goal is to predict one of two possible outcomes.
The model calculates the probability of an input belonging to one of the two classes using a logistic function, and assigns the input to the class with the higher probability

In [61]:
# LogisticRegression model
param_grid = {
        'penalty': ['l1', 'l2'],
        'C': [0.01, 0.1, 1.0, 10.0, 100.0],
        'solver': ['lbfgs', 'newton-cg', 'liblinear'],
        'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 3}, {0: 1, 1: 5}],
        'random_state': [41, 42]
}

# finding hyperparameters for male and best estimator for f1_score
lr = LogisticRegression(max_iter=1000)
grid_search = GridSearchCV(lr, param_grid, cv=kf, scoring=f1_scorer, verbose=3, n_jobs=-1)
grid_search.fit(X_male,y_male)
best_lr_male = grid_search.best_estimator_
f1_score_male = cross_val_score(best_lr_male, X_male,y_male, scoring = make_scorer(f1_score), cv=10).mean()

# finding hyperparameters for female and best estimator for f1_score
grid_search.fit(X_female,y_female)
best_lr_female = grid_search.best_estimator_
f1_score_female = cross_val_score(best_lr_female, X_female,y_female, scoring = make_scorer(f1_score), cv=10).mean()

f1_avarage = (f1_score_male + f1_score_female)/2

print("f1 male: ", f1_score_male)
print("f1 female: ", f1_score_female)
print("f1 average: ", f1_avarage)

Fitting 5 folds for each of 180 candidates, totalling 900 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=LogisticRegression(max_iter=1000), n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1.0, 10.0, 100.0],
                         'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 3},
                                          {0: 1, 1: 5}],
                         'penalty': ['l1', 'l2'], 'random_state': [41, 42],
                         'solver': ['lbfgs', 'newton-cg', 'liblinear']},
             scoring=make_scorer(f1_score, average=macro), verbose=3)

Fitting 5 folds for each of 180 candidates, totalling 900 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=LogisticRegression(max_iter=1000), n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1.0, 10.0, 100.0],
                         'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 3},
                                          {0: 1, 1: 5}],
                         'penalty': ['l1', 'l2'], 'random_state': [41, 42],
                         'solver': ['lbfgs', 'newton-cg', 'liblinear']},
             scoring=make_scorer(f1_score, average=macro), verbose=3)

f1 male:  0.8876334595400446
f1 female:  0.592951895003545
f1 average:  0.7402926772717948


SVC model stands for Support Vector Classification, and it's a type of supervised learning algorithm that can be used for classification tasks.
SVC's main goal is to find the best hyperplane that maximizes the margin, or the distance, between the classes. The hyperplane is a decision boundary that separates the data points into different classes. SVC aims to find the widest possible margin.

In [62]:
# SVC model
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['poly', 'rbf', 'sigmoid'],
    'degree': [2,4],
    'gamma': ['scale', 'auto']
}

# finding hyperparameters for male and best estimator for f1_score
svm = SVC(random_state=42)
grid_search = GridSearchCV(svm, param_grid, cv=kf, scoring=f1_scorer, verbose=3, n_jobs=-1)
grid_search.fit(X_male,y_male)
best_svm_male = grid_search.best_estimator_
f1_score_male = cross_val_score(best_svm_male, X_male,y_male, scoring =f1_scorer, cv=10).mean()

# finding hyperparameters for female and best estimator for f1_score
grid_search.fit(X_female,y_female)
best_svm_female = grid_search.best_estimator_
f1_score_female = cross_val_score(best_svm_female, X_female,y_female, scoring = f1_scorer, cv=10).mean()

f1_avarage = (f1_score_male + f1_score_female)/2

print("f1 male: ", f1_score_male)
print("f1 female: ", f1_score_female)
print("f1 average: ", f1_avarage)


Fitting 5 folds for each of 36 candidates, totalling 180 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=SVC(random_state=42), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10], 'degree': [2, 4],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['poly', 'rbf', 'sigmoid']},
             scoring=make_scorer(f1_score, average=macro), verbose=3)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=SVC(random_state=42), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10], 'degree': [2, 4],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['poly', 'rbf', 'sigmoid']},
             scoring=make_scorer(f1_score, average=macro), verbose=3)

f1 male:  0.6399543040965179
f1 female:  0.6399543040965179
f1 average:  0.6399543040965179


Multinomial Naive Bayes is a type of Naive Bayes algorithm that is used for classification tasks, particularly for text classification.
It's a probabilistic model that is based on Bayes' theorem, which describes the probability of an event based on prior knowledge of conditions that might be related to the event.

In [63]:
#Naive Base model
param_grid = {
    'alpha': [0.01, 0.1, 1.0, 10.0, 100.0],
    'fit_prior': [True, False],
    'class_prior': [None, [0.2, 0.8], [0.3, 0.7], [0.4, 0.6], [0.5, 0.5], [0.6, 0.4], [0.7, 0.3], [0.8, 0.2], [0.9, 0.1]]
}

# finding hyperparameters for male and best estimator for f1_score
nb = MultinomialNB()
grid_search = GridSearchCV(nb, param_grid, cv=kf, scoring=f1_scorer, verbose=3, n_jobs=-1)
grid_search.fit(X_male,y_male)
best_nb_male = grid_search.best_estimator_
f1_score_male = cross_val_score(best_nb_male, X_male,y_male, scoring = make_scorer(f1_score), cv=10).mean()

# finding hyperparameters for female and best estimator for f1_score
grid_search.fit(X_female,y_female)
best_nb_female = grid_search.best_estimator_
f1_score_female = cross_val_score(best_nb_female, X_female,y_female, scoring = make_scorer(f1_score), cv=10).mean()

f1_avarage = (f1_score_male + f1_score_female)/2

print("f1 male: ", f1_score_male)
print("f1 female: ", f1_score_female)
print("f1 average: ", f1_avarage)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=MultinomialNB(), n_jobs=-1,
             param_grid={'alpha': [0.01, 0.1, 1.0, 10.0, 100.0],
                         'class_prior': [None, [0.2, 0.8], [0.3, 0.7],
                                         [0.4, 0.6], [0.5, 0.5], [0.6, 0.4],
                                         [0.7, 0.3], [0.8, 0.2], [0.9, 0.1]],
                         'fit_prior': [True, False]},
             scoring=make_scorer(f1_score, average=macro), verbose=3)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=MultinomialNB(), n_jobs=-1,
             param_grid={'alpha': [0.01, 0.1, 1.0, 10.0, 100.0],
                         'class_prior': [None, [0.2, 0.8], [0.3, 0.7],
                                         [0.4, 0.6], [0.5, 0.5], [0.6, 0.4],
                                         [0.7, 0.3], [0.8, 0.2], [0.9, 0.1]],
                         'fit_prior': [True, False]},
             scoring=make_scorer(f1_score, average=macro), verbose=3)

f1 male:  0.8595085426993414
f1 female:  0.5627700455444358
f1 average:  0.7111392941218886


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

After defining the pre-processing steps and finding an optimal model based on the problem at hand, we manualy select the best-preforming model and use it to make a prediction over the test dataset.

In [64]:
# preprocess the test data
X_test = vec.transform(df_test.story)
X_test_CV = pd.DataFrame(X_test.toarray(), columns=vec.get_feature_names_out())

# use the best logistic regression model to make predictions
y_pred = best_lr_male.predict(X_test)
y_pred = ['m' if pred == 1 else 'f' for pred in y_pred]

df_predicted = pd.DataFrame({'test_example_id': df_test.test_example_id, 'predicted_gender': y_pred})
df_predicted.head()
df_predicted.tail()

,test_example_id,predicted_gender
0,0,m
1,1,m
2,2,m
3,3,m
4,4,f


,test_example_id,predicted_gender
318,318,m
319,319,m
320,320,m
321,321,m
322,322,m


In [65]:
df_predicted.to_csv("C:/Users/tomer/Desktop/תואר ראשון/שנה ב/למידת מכונה/classification_results.csv",index=False)